In [1]:
## fine tuning sam2

In [1]:
from mb_annotation.segsam2 import load_data,sam_predictor,train_model

/home/winnow/.local/lib/python3.10/site-packages/sam2/modeling/sam/transformer.py:23: UserWarning: Flash Attention is disabled as it requires a GPU with Ampere (8.0) CUDA capability.
  OLD_GPU, USE_FLASH_ATTN, MATH_KERNEL_ON = get_sdpa_settings()


In [2]:
data = load_data("/home/winnow/Desktop/data_to_test/final_food_data.csv")
data[0]

{'image': '/home/winnow/Desktop/data_to_test/images/0000.png',
 'annotation': '/home/winnow/Desktop/data_to_test/mask/0000_mask.png'}

In [3]:
predictor = sam_predictor('/home/winnow/segment-anything-2/checkpoints/sam2_hiera_small.pt','./sam2_hiera_s.yaml',device='cpu')

In [4]:
new_predictor = train_model(data, predictor, epochs=5, lr=1e-6,device='cpu',save_step=5)

step:0 loss:2.6383 seg_loss:2.5943 score_loss:0.8800 mean_iou:0.0008
step:1 loss:0.2170 seg_loss:0.2127 score_loss:0.0862 mean_iou:0.0023


In [ ]:
import torch

sam2_checkpoint = "/home/winnow/segment-anything-2/checkpoints/sam2_hiera_small.pt"
model_cfg = "./sam2_hiera_s.yaml" # "sam2_hiera_l.yaml"

predictor = sam_predictor( sam2_checkpoint, model_cfg,device="cuda")
predictor.model.load_state_dict(torch.load("./sam_model_checkpoints/sam_model.pt"))

In [ ]:
import cv2
import numpy as np
def read_image(image_path, mask_path): # read and resize image and mask
        img = cv2.imread(image_path)[...,::-1]  # read image
        mask = cv2.imread(mask_path,0)
        r = np.min([1024 / img.shape[1], 1024 / img.shape[0]])
        img = cv2.resize(img, (int(img.shape[1] * r), int(img.shape[0] * r)))
        mask = cv2.resize(mask, (int(mask.shape[1] * r), int(mask.shape[0] * r)),interpolation=cv2.INTER_NEAREST)
        return img, mask


def get_points(mask,num_points): # Sample points inside the input mask
        points=[]
        for i in range(num_points):
            coords = np.argwhere(mask > 0)
            yx = np.array(coords[np.random.randint(len(coords))])
            points.append([[yx[1], yx[0]]])
        return np.array(points)

In [ ]:
# read image and sample points
image,mask = read_image('/home/winnow/Desktop/data_to_test/images/0003.png','/home/winnow/Desktop/data_to_test/mask/0003_mask.png')
input_points = get_points(mask,50)

In [ ]:
import matplotlib.pyplot as plt

with torch.no_grad():
        predictor.set_image(image)
        masks, scores, logits = predictor.predict(
            point_coords=input_points,
            point_labels=np.ones([input_points.shape[0],1])
        )

# Short predicted masks from high to low score

np_masks = np.array(masks[:,0]) # remove .cpu().numpy() as masks is already a numpy array
np_scores = scores[:,0]
shorted_masks = np_masks[np.argsort(np_scores)][::-1]


# Stitch predicted mask into one segmentation mask

seg_map = np.zeros_like(shorted_masks[0],dtype=np.uint8)
occupancy_mask = np.zeros_like(shorted_masks[0],dtype=bool)
for i in range(shorted_masks.shape[0]):
    mask = shorted_masks[i]
    if (mask*occupancy_mask).sum()/mask.sum()>0.15: continue
    mask[occupancy_mask]=0
    # Convert mask to boolean for indexing
    seg_map[mask > 0.5] = i + 1  # Assuming 0.5 as the threshold for positive prediction
    occupancy_mask[mask > 0.5] = 1


# create colored annotation map
height, width = seg_map.shape

# Create an empty RGB image for the colored annotation
rgb_image = np.zeros((height, width, 3), dtype=np.uint8)
# Map each class number to a random  color


for id_class in range(1,seg_map.max()+1):
    rgb_image[seg_map == id_class] = [np.random.randint(255), np.random.randint(255), np.random.randint(255)]

# save and display

cv2.imwrite("annotation.png",rgb_image)
cv2.imwrite("mix.png",(rgb_image/2+image/2).astype(np.uint8))

plt.imshow(rgb_image)
# plt.imshow(image)
plt.show()

cv2.waitKey()